# PySpark Recap

- **Spark** is an analytics engine that allows queries to be completed by multiple machines (a "cluster") in parallel.
- This makes it good for dealing with big data
- PySpark is an API that enables us to use Spark with python code

In [ ]:
!pip install pyspark

# Import PySpark functions

PySpark comes with a library of functions we'll need to use in our code, so we'll import these first:

In [ ]:
from pyspark.sql import functions as F

# The SparkSession

In Spark, there is a special object called the SparkSession. This is the entry point to Spark's functionality. When you use Databricks, it creates this for you automatically. But outside Databricks, we have to create it ourselves:

In [ ]:
from pyspark import sql

spark = (sql.SparkSession
    .builder
    .appName("pyspark_intro")
    .getOrCreate()
)

# Download Artificial HES data

We will use the Artificial NHS Hospital Episode Statistics Accident and Emergency (HES AE) data from 2003 for these examples. The cell below just downloads this data from the public website, and unzips the CSVs inside it.

In [ ]:
# These libraries will help us download the file
import zipfile
import io
from pathlib import Path
import requests

zip_file_url = "https://files.digital.nhs.uk/assets/Services/Artificial%20data/Artificial%20HES%20final/artificial_hes_ae_202302_v1_sample.zip"
path_to_downloaded_data = "inputs/artificial_hes/artificial_hes_ae_202302_v1_sample.zip/artificial_hes_ae_202302_v1_sample/artificial_hes_ae_2122.csv"

filename = Path(zip_file_url).name
output_path = f"inputs/artificial_hes/{filename}"

response = requests.get(zip_file_url, stream=True,timeout=3600)
downloaded_zip = zipfile.ZipFile(io.BytesIO(response.content))
downloaded_zip.extractall(output_path)

# DataFrames

A DataFrame is a tabular data format, stored in memory. We can read a CSV into a DataFrame like this:

In [ ]:
df_hes = (spark.read
    .option('header', 'true')
    .csv(path_to_downloaded_data)
)

# Displaying data

In Databricks you can use the `display()` function and pass the DataFrame to it. This gives a nice tabular output where you can look at data.

It can take a while with a lot of data or intense queries!

Outside of Databricks we use `df.show()` like this:

In [ ]:
df_hes.show()

# Manipulating Data

When you create a DataFrame object, it automatically has access to a range of functions you can use to transform an manipulate the data. Because these functions are attached to an object, we call them "methods".

These DataFrame methods, along with the functions we imported using the `from pyspark.sql import functions as F` are what we'll use to work with data in PySpark.

We went over quite a few of these last time. Let's recap some of the main ones.

# SELECT

In [ ]:
df_hes_filtered = (df_hes
    .select(
      "EPIKEY",
      "CCG_GP_PRACTICE",
      "ARRIVALDATE"
    )             
)

df_hes_filtered.show()

- We have used the `.select()` method on `df_hes` - this returns a new DataFrame with only the columns we specified.
- We stored this new df in a variable called `df_hes_filtered`. The original DataFrame, `df_hes`, is unchanged.
- Then we used the `.show()` method to see the results of our query.

# ORDER BY

In [ ]:
df_hes_filtered = (df_hes
    .select(
      "EPIKEY",
      "CCG_GP_PRACTICE",
      "ARRIVALDATE"
    )
    .orderBy("ARRIVALDATE")
)

df_hes_filtered.show()

It's ascending by default. For descending you could use:

In [ ]:
df_hes_filtered = (df_hes
    .select(
      "EPIKEY",
      "CCG_GP_PRACTICE",
      "ARRIVALDATE"
    )
    .orderBy( F.desc("ARRIVALDATE") )
)

df_hes_filtered.show()

Remember: The `F` in `F.desc()` indicates that it's a function from the pyspark.sql.functions library that we imported at the top. So it's not a DataFrame method.

# WHERE

In [ ]:
df_hes_filtered = (df_hes
    .select(
      "EPIKEY",
      "CCG_GP_PRACTICE",
      "ARRIVALDATE"
    )
    .where( F.col("ARRIVALDATE") > "2021-06-01")
    .orderBy("ARRIVALDATE")
)

df_hes_filtered.show()

Note how we are chaining the DataFrame methods one after another. This is how you build queries in PySpark.

# GROUP BY / COUNT(*)

In [ ]:
df_hes_filtered = (df_hes
    .where(
        (F.col("ARRIVALDATE") > "2021-06-01")
        |
        (F.col("ARRIVALDATE").isNull())
    )
    .groupBy(
      "CCG_GP_PRACTICE"
    )
    .count()
    .orderBy(F.desc('count'))
)

df_hes_filtered.show()